In [1]:
from dotenv import dotenv_values
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


LANGCHAIN_TRACING_V2="true"

config = dotenv_values(".env")


USER_AGENT environment variable not set, consider setting it to identify your requests.


ModuleNotFoundError: No module named 'langchain.chains'

In [2]:
llm = ChatOpenAI(api_key=config["OPEN_AI_KEY"])

llm.invoke("How can LangSmith help with testing")

AIMessage(content='LangSmith can help with testing by providing a platform for creating and executing automated test scripts. This can help streamline the testing process by allowing testers to quickly and efficiently run tests on various aspects of the software or application. LangSmith also offers features such as test case management, test data management, and reporting, which can help testers track their progress and identify any issues or bugs that arise during testing. Additionally, LangSmith can integrate with other testing tools and systems, making it easy for testers to collaborate and share their results with other team members. Overall, LangSmith can help improve the efficiency and effectiveness of the testing process, ultimately leading to a higher quality product.', response_metadata={'token_usage': {'completion_tokens': 132, 'prompt_tokens': 14, 'total_tokens': 146}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs':

In [3]:
# LLM Chains

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

In [4]:
chain = prompt | llm

In [5]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='Langsmith can help with testing in several ways:\n\n1. **Automated Testing:** Langsmith can generate automated test scripts to validate the functionality of software applications. By using Langsmith to automatically generate test cases based on predefined criteria, testers can save time and effort in writing test scripts manually.\n\n2. **Test Data Generation:** Langsmith can create synthetic test data sets to be used in testing scenarios. This can help testers cover a wider range of test cases and edge conditions, ensuring thorough testing coverage.\n\n3. **Exploratory Testing Support:** Langsmith can assist testers in exploring different paths and scenarios within an application by generating test cases dynamically. This can help testers uncover hidden defects and improve overall test coverage.\n\n4. **Regression Testing:** Langsmith can automate the execution of regression test suites, allowing testers to quickly validate that new code changes have not introduced 

In [6]:
output_parser = StrOutputParser()

In [7]:
chain = prompt | llm | output_parser

In [8]:
chain.invoke({"input": "how can langsmith help with testing"})

"Langsmith can help with testing in several ways, including:\n\n1. **Automated Testing:** Langsmith can generate test scripts or test data to automate the testing process. This can save time and effort compared to manual testing.\n\n2. **Load Testing:** Langsmith can simulate a large number of users accessing a system simultaneously to test its performance under heavy loads.\n\n3. **Security Testing:** Langsmith can help identify security vulnerabilities by generating test cases that simulate potential attacks or breaches.\n\n4. **Regression Testing:** Langsmith can help ensure that new code changes do not introduce unintended side effects by generating test cases to verify the system's behavior against previous versions.\n\n5. **Data Testing:** Langsmith can generate test data to ensure that the system functions correctly with various types of input data.\n\nOverall, Langsmith can enhance the efficiency and effectiveness of testing processes by automating repetitive tasks and providin

In [9]:
# Retrieval Chains

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [10]:
# Next, we need to index it into a vectorstore. This requires a few components, namely an embedding model and a vectorstore.

embeddings = OpenAIEmbeddings(api_key=config["OPEN_AI_KEY"])

In [11]:
# Now, we can use this embedding model to ingest documents into a vectorstore. We will use a simple local vectorstore, FAISS, for simplicity's sake.

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [12]:

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [13]:
# If we wanted to, we could run this ourselves by passing in documents directly:

from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing users to visualize test results.'

In [14]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "what is the Prompt Hub?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'There is no information provided in the context about the Prompt Hub, so it is not possible to answer the question based on the given context.'

In [15]:
# However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [16]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith can help with testing by allowing developers to create datasets, run tests on LLM applications, upload test cases in bulk, create custom evaluations, compare results for different configurations, provide a playground environment for rapid iteration and experimentation, enable beta testing to collect real-world performance data, capture feedback from users, annotate traces with different criteria, add runs to datasets to refine and improve performance, and monitor key metrics over time for debugging and A/B testing purposes.


In [17]:
response = retrieval_chain.invoke({"input": "where can I access the public view of the hub? share the url please"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

Based on the provided context, the public view of the hub can be accessed at the following URL: https://langsmith.com/hub


In [18]:
# Conversation Retrieval Chain

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [19]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing new LLM applications, we suggest having LangSmith tracing enabled

In [20]:
# Create a new chain to continue the conversation with these retrieved documents

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [21]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug wh

In [22]:
# Agent

from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [23]:
# Can add more tools to be used

tools = [retriever_tool]

In [25]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# You need to set OPENAI_API_KEY environment variable or pass it as argument `api_key`.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key=config["OPEN_AI_KEY"])
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [26]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith testing'}`


LangSmith User Guide | 🦜️🛠️ LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping​Prototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.
The ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly im

{'input': 'how can langsmith help with testing?',
 'output': "LangSmith can help with testing in various ways during the application development lifecycle. Here are some key ways LangSmith supports testing:\n\n1. **Prototyping**: LangSmith allows for quick experimentation between prompts, model types, retrieval strategies, and other parameters to understand how the model is performing during the prototyping phase.\n\n2. **Debugging**: LangSmith provides tracing capabilities that give clear visibility and debugging information at each step of an LLM sequence, making it easier to identify and root-cause issues when things go wrong.\n\n3. **Initial Test Set**: Developers can create datasets of inputs and reference outputs to run tests on LLM applications. LangSmith makes it easy to run custom evaluations to score test results.\n\n4. **Comparison View**: LangSmith offers a user-friendly comparison view for test runs to track and diagnose regressions in test scores across multiple revisions

In [27]:
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...
I'm sorry, but I am not able to provide real-time weather information. You can check the weather in San Francisco by using a weather website or app like Weather.com or AccuWeather.

> Finished chain.


{'input': 'what is the weather in SF?',
 'output': "I'm sorry, but I am not able to provide real-time weather information. You can check the weather in San Francisco by using a weather website or app like Weather.com or AccuWeather."}

In [28]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LLM application testing services'}`


Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping​Prototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.
The ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.D

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': 'LangSmith provides a platform for LLM application development, monitoring, and testing. Here are some ways LangSmith can help with testing your LLM applications:\n\n1. Prototyping: LangSmith supports quick experimentation between prompts, model types, retrieval strategy, and other parameters for prototyping LLM applications.\n\n2. Debugging: LangSmith provides tracing capabilities to help developers debug issues in LLM applications by giving clear visibility and debugging information at each step of an LLM sequence.\n\n3. Initial Test Set: Developers can create datasets of inputs and reference outputs to run tests on LLM applications. LangSmith allows for running custom evaluations to score test results.\n\n4. Comparison View: LangSmith offers a comparison view for test runs to track and diagnose regressions in test scores across mul